In [1]:
from torch.utils.data import DataLoader
from src.dataset import FrameDataset, custom_collate
from sklearn.preprocessing import LabelEncoder
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch
from src.averager import Averager
import os


from src import config
hparams = config.hparams

In [2]:
le = LabelEncoder()
le.fit(['apadding', 'bicycle','human', 'motorcycle', 'vehicle'])


LabelEncoder()

In [7]:
training_data = FrameDataset(root = hparams.train_path, label_encoder = le)
train_dataloader = DataLoader(training_data, batch_size=5, shuffle=True, collate_fn=custom_collate)

In [3]:
# load a model; pre-trained on COCO
model = fasterrcnn_resnet50_fpn(num_classes=hparams.num_classes)

In [4]:
device = torch.device(hparams.device)

In [5]:
# initialize logging
loggers = []
log_name = f"{hparams.experiment_name}-{hparams.num_classes}"

# make sure the output directory exists
output_dir = os.path.join(hparams.output_dir,log_name)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if 'tensorboard' in hparams.loggers:
    from pytorch_lightning.loggers import TensorBoardLogger
    loggers.append(TensorBoardLogger(save_dir='tb_logs/', name=log_name, default_hp_metric=False))

In [8]:
from pytorch_lightning import Trainer
trainer = Trainer(accelerator=hparams.accelerator,
    max_epochs=hparams.max_epochs,
    logger=loggers
)

trainer.fit(model, train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `FasterRCNN`

In [6]:
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.00001)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
lr_scheduler = None

num_epochs = 5

loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for frames, targets in train_dataloader:
        
        frames = [frame.to(device) for frame in frames]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(frames, targets)   ##Return the loss

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)  #Average out the loss

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")

Iteration #50 loss: 0.9948012961930723
Iteration #100 loss: 0.9469753066016935
Iteration #150 loss: 0.415995321098141
Iteration #200 loss: 0.41657329001827836
Iteration #250 loss: 0.37250390882477746
Iteration #300 loss: 0.21201269374825674
Iteration #350 loss: 0.26147053919509766
Iteration #400 loss: 0.3789270970820444
Iteration #450 loss: 0.2628116381492644
Iteration #500 loss: 0.8007043066721005
Iteration #550 loss: 0.5293398501731628
Iteration #600 loss: 0.5715343014869576
Iteration #650 loss: 0.6960379809678382
Iteration #700 loss: 0.6069735954553439
Iteration #750 loss: 0.30042116045238826
Iteration #800 loss: 0.4375571932864135
Iteration #850 loss: 0.4630438817880579
Iteration #900 loss: 0.504999027884293
Epoch #0 loss: 0.5311573010617584
Iteration #950 loss: 0.1910115289601049
Iteration #1000 loss: 0.7373331665307973


KeyboardInterrupt: 